# Rf 5 1 3_Wsfactory_Tools
Organization and simultaneous fits: RooCustomizer and RooSimWSTool interface in factory workspace tool in a complex
standalone B physics example



**Author:** 04/2009 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:57 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooWorkspace.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [3]:
RooWorkspace *w = new RooWorkspace("w");


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Build a complex example p.d.f.
 -----------------------------------------------------------

Make signal model for cpv: a bmixing decay function in t (convoluted with a triple gaussian resolution model)
                            times a Gaussian function the reconstructed mass

In [4]:
w->factory("PROD::sig(  BMixDecay::sig_t( dt[-20,20], mixState[mixed=1,unmix=-1], tagFlav[B0=1,B0bar=-1], "
           "tau[1.54], dm[0.472], w[0.05], dw[0],"
           "AddModel::gm({GaussModel(dt,biasC[-10,10],sigmaC[0.1,3],dterr[0.01,0.2]),"
           "GaussModel(dt,0,sigmaT[3,10]),"
           "GaussModel(dt,0,20)},{fracC[0,1],fracT[0,1]}),"
           "DoubleSided ),"
           "Gaussian::sig_m( mes[5.20,5.30], mB0[5.20,5.30], sigmB0[0.01,0.05] ))");

Make background component: a plain decay function in t times an argus function in the reconstructed mass

In [5]:
w->factory("PROD::bkg(  Decay::bkg_t( dt, tau, gm, DoubleSided),"
           "ArgusBG::bkg_m( mes, 5.291, k[-100,-10]))");

Make composite model from the signal and background component

In [6]:
w->factory("SUM::model( Nsig[5000,0,10000]*sig, NBkg[500,0,10000]*bkg )");

Example of roosimwstool interface
 ------------------------------------------------------------------

Introduce a flavour tagging category tagcat as observable with 4 states corresponding
 to 4 flavour tagging techniques with different performance that require different
 parameterizations of the fit model

 RooSimWSTool operation:
     - Make 4 clones of model (for each tagCat) state, that will gain an individual
       copy of parameters w,dw and biasC. The other parameters remain common
     - Make a simultaneous p.d.f. of the 4 clones assigning each to the appropriate
       state of the tagCat index category

Roosimwstool is interfaced as meta-type simclone in the factory. the $splitparam()
 argument maps to the SplitParam() named argument in the RooSimWSTool constructor

In [7]:
w->factory("SIMCLONE::model_sim( model, $SplitParam({w,dw,biasC},tagCat[Lep,Kao,NT1,NT2]))");

Example of roocustomizer interface
 -------------------------------------------------------------------

 Class RooCustomizer makes clones of existing p.d.f.s with certain prescribed
 modifications (branch of leaf node replacements)

 Here we take our model (the original before RooSimWSTool modifications)
 and request that the parameter w (the mistag rate) is replaced with
 an expression-based function that calculates w in terms of the Dilution
 parameter D that is defined as D = 1-2*w

Make a clone model_d of original 'model' replacing 'w' with 'expr('0.5-d/2',d[0,1])'

In [8]:
w->factory("EDIT::model_D(model, w=expr('0.5-D/2',D[0,1]) )");

Print workspace contents

In [9]:
w->Print();


RooWorkspace(w) w contents

variables
---------
(D,NBkg,Nsig,biasC,biasC_Kao,biasC_Lep,biasC_NT1,biasC_NT2,dm,dt,dterr,dw,dw_Kao,dw_Lep,dw_NT1,dw_NT2,fracC,fracT,k,mB0,mes,mixState,sigmB0,sigmaC,sigmaT,tagCat,tagFlav,tau,w,w_Kao,w_Lep,w_NT1,w_NT2)

p.d.f.s
-------
RooProdPdf::bkg[ bkg_t * bkg_m ] = 0.307193
RooProdPdf::bkg_Kao[ bkg_t_Kao * bkg_m ] = 0.307193
RooProdPdf::bkg_Lep[ bkg_t_Lep * bkg_m ] = 0.307193
RooProdPdf::bkg_NT1[ bkg_t_NT1 * bkg_m ] = 0.307193
RooProdPdf::bkg_NT2[ bkg_t_NT2 * bkg_m ] = 0.307193
RooArgusBG::bkg_m[ m=mes m0=5.291 c=k p=0.5 ] = 0.279062
RooDecay::bkg_t[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_Kao[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_Lep[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_NT1[ t=dt tau=tau ] = 1.10081
RooDecay::bkg_t_NT2[ t=dt tau=tau ] = 1.10081
RooAddPdf::model[ Nsig * sig + NBkg * bkg ] = 1.88229
RooAddPdf::model_D[ Nsig * sig_model_D + NBkg * bkg ] = 1.5029
RooAddPdf::model_Kao[ Nsig * sig_Kao + NBkg * bkg_Kao ] = 1.88229
RooAddPdf::model

Make workspace visible on command line

In [10]:
gDirectory->Add(w);